#**MNIST- refers to image identification of hand written digits data**

**This project focuses on creating a neural netwrok model completly using numpy instead pof tensorflow library**

As for the data set we can either get it through kaggle, keras, or even openml
In this project we are using openML since we do not want to use keras in this complete code

##Some conventions
Upper case letters refres to 2D matrix datas. Ex- shape( samples, features )
Lower case refrs to 1D vectors. Ex- shape(label)

In [3]:

import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

#Load and seperate data for train and test

The data will be a 28x28 pixel image's greyscale intensity data (0-255 for each pixel) for each number which is filled with  black to white colors, which we refer as 0-1. Where 0 refers to black and 1 refers to white, but in system recoginition the value is given as its density value which is 0 for black and 255 for white, hence to use the data set we have to do these steps

First, the openml data is given as 
X is a float of 784 (28x28) values per row that is 784 features for each sample, and y is a list of strings - ('0','1',2'....,'9')
hence we convert y label values to int from string
then divide then x data by 255 to get values between 0 and 1.
(x value is color that is 01 and y value is 0-9 number labels)

In [4]:
#Load MNIST data from openML
X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)
#Convert string to int
y = y.astype(np.int64)
#Normalize pixel value that is 0 to 255 is changed to 0-1
X = X / 255.0
#split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=10000, random_state=42, stratify=y)
#In above, random_state=42 refers to random splitting for better working, the 42 can be any number
# stratify = y is used to arrange y values such that it is not too biased in either test or train set ( that is the same label (0-9) is spread evenly in percentage)

print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")
#this data set has 60k sample each of 784px hence those many features for each, test data set is 10k as given above 

Train shape: (60000, 784), Test shape: (10000, 784)


#Batch Generator
###Neural network do not train on the entire dataset at once - too slow and too much memory
INSTEAD WE USE MINI BATCHES, taht is train 32-64 data per batch and then take another batch while removing the previous batch from emory - this helps when the dataset is large.

ex: when we have 60k images we train 32 to 64 images per batch

In [ ]:
def batch_generator(X, y, batch_size=32):
    n_samples = X.shape[0] #total number of samples
    indices = np.arange(n_samples)   # this creates a list with the specified number of terms like [0,1,2,3,...]
    np.random.shuffle(indices)       # shuffle the list for randomness
    
    for start in range(0, n_samples, batch_size):
        end = start + batch_size
        batch_ids = indices[start:end]
        yield X[batch_ids], y[batch_ids] #yield is used such that memory is saved since it outputs one step then forgets memory and does it for the next batch unlike how return does is send it as a largee single block
#for those with doubts how X[[23,45,67,32,43,..]] this type of data gives output - this works as X[23],X[45],... note that this is how np.array works and will noyt work with python lists

#Layer Dense

For this we will be using class instead of direct functions

##For those who do not have any basic idea of class-
in simple words it is where we group multiple functions all having connected variable for each object (child of the class) created/initialized
the first function called __init__ runs eventhough it is not called whenever an object is defined, meaning it initialises the data such as the basic variables that are used by that object's functions
the functions inside the object are accessed by '.' operator

##Why use a class for dense
A Dense (fully connected) layer needs:
- A set of **weights** and **biases** for each layer.
- A **forward method** to compute outputs.
- A **backward method** to compute gradients (for training).


In [ ]:
#object = DenseLayer(n_iputs, n_neurons) these parameter values are the __init__ function's parameter
class DenseLayer:
    def __init__(self, n_inputs, n_neurons):
        """
        A fully connected (Dense) layer.
        n_inputs: number of features (e.g. 784 for MNIST)
        n_neurons: number of neurons/units in this layer
        """
        # Initialize weights small random values
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)#creates a matrix of random numbers as value which is multipled by 0.01 to reduce its size, the matrix shape is (n_iputs,n_neurons)
        # matrix shape is (784 features which has one weight each,number of units in this layer)
        # Initialize biases as zeros
        self.biases = np.zeros((1, n_neurons)) #creates a array of zeroes for the guven shape (1,number of units in the layer)

    def forward(self, inputs):
        """
        Forward pass through the layer.
        inputs: shape (batch_size, n_inputs)
        """
        self.inputs = inputs  # keep for backward pass later
        self.output = np.dot(inputs, self.weights) + self.biases

IndexError: too many indices for array: array is 1-dimensional, but 5 were indexed